# Input Secret

In [30]:
# secret = input("What do you want to hide?")
secret = "Hello World"
print(f"Your secret: {secret}")

Your secret: Hello World


# Convert Secret into Binary

In [112]:
secret_in_binary = ''.join(format(ord(char), '08b') for char in secret)
print("Binary representation of your secret: " + ' '.join(format(ord(char), '08b') for char in secret))

print(f"The secret is {len(secret_in_binary)} bits long.")

def find_factor(n):
  factor_values = []
  for i in range(1, n + 1):
    if n % i == 0:
      factor_values.append(i)
  values = ""
  for v in factor_values:
    values += str(v) + ", "
  return values

factors = find_factor(len(secret_in_binary))
factors = factors[:-2]
# factor = int(input(f"We can choose from these factors for the length of our binary sequence: {factors}"))
factor = 8
print(f"Our binary sequence (ID of each image) will be {factor} bits long.")

Binary representation of your secret: 01001000 01100101 01101100 01101100 01101111 00100000 01010111 01101111 01110010 01101100 01100100
The secret is 88 bits long.
Our binary sequence (ID of each image) will be 8 bits long.


# Generate Images

In [69]:
import os

folder_path = "./cs4485-images"

file_names = []
for file_name in os.listdir(folder_path):
   file_path = os.path.join(folder_path, file_name)
   if os.path.isfile(file_path):
     file_names.append(file_name)

file_names = sorted(file_names, key=lambda x: (int(x.split('_')[1].replace('prompt', '')), int(x.split('_')[3].replace('image', '').replace('.png', ''))))

print(file_names)

#get image shape
# import numpy as np
# import cv2
# image = cv2.imread(f"./cs4485-images/{file_names[255]}")
# print(f"Image shape: {image.shape}")

['prompt_0_image_0.png', 'prompt_0_image_1.png', 'prompt_0_image_2.png', 'prompt_0_image_3.png', 'prompt_0_image_4.png', 'prompt_0_image_5.png', 'prompt_0_image_6.png', 'prompt_0_image_7.png', 'prompt_0_image_8.png', 'prompt_0_image_9.png', 'prompt_0_image_10.png', 'prompt_0_image_11.png', 'prompt_0_image_12.png', 'prompt_0_image_13.png', 'prompt_0_image_14.png', 'prompt_0_image_15.png', 'prompt_1_image_0.png', 'prompt_1_image_1.png', 'prompt_1_image_2.png', 'prompt_1_image_3.png', 'prompt_1_image_4.png', 'prompt_1_image_5.png', 'prompt_1_image_6.png', 'prompt_1_image_7.png', 'prompt_1_image_8.png', 'prompt_1_image_9.png', 'prompt_1_image_10.png', 'prompt_1_image_11.png', 'prompt_1_image_12.png', 'prompt_1_image_13.png', 'prompt_1_image_14.png', 'prompt_1_image_15.png', 'prompt_2_image_0.png', 'prompt_2_image_1.png', 'prompt_2_image_2.png', 'prompt_2_image_3.png', 'prompt_2_image_4.png', 'prompt_2_image_5.png', 'prompt_2_image_6.png', 'prompt_2_image_7.png', 'prompt_2_image_8.png', 'pr

# Prompt Mapping

In [107]:
# pm = ["{0:04b}".format(int(name.split('_')[1].replace('prompt', ''))) for name in file_names]
# print(pm)
# print(f"Length of pm array: {len(pm)}")

pm = []
for i in range(len(file_names) // 16):  # Assuming 16 images per prompt
    binary_code = f"{i:04b}"
    for j in range(16):
        file_name = f"prompt_{i}_image_{j}.png"  # Construct file name
        pm.append((file_name, binary_code))

print(pm)
print(f"Length of pm array: {len(pm)}") 

[('prompt_0_image_0.png', '0000'), ('prompt_0_image_1.png', '0000'), ('prompt_0_image_2.png', '0000'), ('prompt_0_image_3.png', '0000'), ('prompt_0_image_4.png', '0000'), ('prompt_0_image_5.png', '0000'), ('prompt_0_image_6.png', '0000'), ('prompt_0_image_7.png', '0000'), ('prompt_0_image_8.png', '0000'), ('prompt_0_image_9.png', '0000'), ('prompt_0_image_10.png', '0000'), ('prompt_0_image_11.png', '0000'), ('prompt_0_image_12.png', '0000'), ('prompt_0_image_13.png', '0000'), ('prompt_0_image_14.png', '0000'), ('prompt_0_image_15.png', '0000'), ('prompt_1_image_0.png', '0001'), ('prompt_1_image_1.png', '0001'), ('prompt_1_image_2.png', '0001'), ('prompt_1_image_3.png', '0001'), ('prompt_1_image_4.png', '0001'), ('prompt_1_image_5.png', '0001'), ('prompt_1_image_6.png', '0001'), ('prompt_1_image_7.png', '0001'), ('prompt_1_image_8.png', '0001'), ('prompt_1_image_9.png', '0001'), ('prompt_1_image_10.png', '0001'), ('prompt_1_image_11.png', '0001'), ('prompt_1_image_12.png', '0001'), ('pr

# Amplitude Mapping

## Weighted Amplitudes

In [92]:
import cv2
import numpy as np
import math

# Constants for FrCHFMs calculation
ORDERS = range(5)  # Orders n (0 to 4)
REPETITIONS = range(5)  # Repetitions m (0 to 4)
FRACTIONAL_PARAMETER = 0.5  # Fractional parameter
IMAGE_SIZE = (512, 512)  # Assuming 512x512 images
EPSILON = 1e-10  # Small constant to avoid division by zero

# Load and preprocess the image
def load_grayscale_image(file_path):
    image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, IMAGE_SIZE)  # Resize to 512x512 if needed
    return image.astype(float)

# Define radial basis function Rtn(r) for FrCHFMs with stability fix
def radial_basis_function_vectorized(r, n, t):
    r[r < EPSILON] = EPSILON  # Avoid division by zero
    factor = np.sqrt(t) * np.power(r, t - 1) * np.sqrt(8 / np.pi)
    polynomial_sum = np.zeros_like(r)
    
    for k in range(n // 2 + 1):
        coefficient = (-1) ** k * math.factorial(n - k) / (math.factorial(k) * math.factorial(n - 2 * k))
        polynomial_sum += coefficient * np.power(4 * np.power(r, t) - 2, n - 2 * k)
    
    return factor * polynomial_sum

# Compute FrCHFMs for a grayscale image
def calculate_frchfms(image, t=FRACTIONAL_PARAMETER):
    rows, cols = image.shape
    center_x, center_y = cols // 2, rows // 2
    
    # Calculate r and theta arrays for the whole image
    x = np.arange(cols) - center_x
    y = np.arange(rows) - center_y
    X, Y = np.meshgrid(x, y)
    r = np.sqrt(X ** 2 + Y ** 2) / (cols // 2)
    theta = np.arctan2(Y, X)

    amplitudes = []
    for n in ORDERS:
        for m in REPETITIONS:
            if n == 0 and m == 0: continue  # Skip the (0,0) case
            # Vectorized computation of radial basis and Fourier component
            radial_value = radial_basis_function_vectorized(r, n, t)
            angular_value = np.exp(-1j * m * theta)
            
            # Compute the FrCHFM by summing the weighted image values
            frchfm = np.sum(image * radial_value * angular_value)
            amplitudes.append(abs(frchfm))
    return amplitudes

# Apply weights to the amplitudes to return weighted amplitudes
def calculate_weighted_amplitudes(amplitudes):
    weights = [(n + 1) + (m + 1) for n in ORDERS for m in REPETITIONS if not (n == 0 and m == 0)]
    weighted_amplitudes = [amplitude * weight for amplitude, weight in zip(amplitudes, weights)]
    return weighted_amplitudes

# Main code to load image and calculate weighted amplitudes
# file_path = "./cs4485-images/prompt_0_image_0.png"  # Update with correct path
# image = load_grayscale_image(file_path)
# amplitudes = calculate_frchfms(image)
# weighted_amplitudes = calculate_weighted_amplitudes(amplitudes)

# print("Weighted 24 amplitudes:", weighted_amplitudes)

WAs = []
for file_name in file_names:
    # print(image)
    file_path = f"./cs4485-images/{file_name}"  # Update with correct path
    image = load_grayscale_image(file_path)
    amplitudes = calculate_frchfms(image)
    weighted_amplitudes = calculate_weighted_amplitudes(amplitudes)
    print(f"{file_name}: {weighted_amplitudes} \n")
    WAs.append(weighted_amplitudes)
    

prompt_0_image_0.png: [np.float64(92446228.83525425), np.float64(114675135.76885301), np.float64(153730802.91607094), np.float64(155943444.06828588), np.float64(43159721.80183371), np.float64(228118384.21785137), np.float64(283447090.4833941), np.float64(323302575.961489), np.float64(455557345.7854928), np.float64(502778040.00010175), np.float64(428215387.1247897), np.float64(529284428.8371862), np.float64(638690915.6877592), np.float64(585693421.1263492), np.float64(339713566.1733609), np.float64(697062285.0685332), np.float64(792174449.1108611), np.float64(879886576.6343734), np.float64(1226937496.5100608), np.float64(1239071870.5364697), np.float64(982342155.7301648), np.float64(1170321439.7613332), np.float64(1364779031.8896039), np.float64(1068426705.4335791)] 

prompt_0_image_1.png: [np.float64(83915293.65848988), np.float64(116153721.76041521), np.float64(138873259.20888588), np.float64(171714082.62327126), np.float64(37155798.94180141), np.float64(226753053.06511736), np.float6

## Euclidean Distance

In [104]:
# Debugging: check structure of weighted_amplitudes_all
# print(f"Number of images processed: {len(WAs)}")
# for i, amplitudes in enumerate(WAs):
#     print(f"Image {i}: Length of amplitudes = {len(amplitudes)}")

# Step 1: Group into 16-image groups (assuming file_names are already sorted)
groups = []
for i in range(16):
    group = WAs[i*16:(i+1)*16]
    groups.append(group)
    
print(groups)

# Step 2: Calculate distances within each group and assign 4-bit codes
am = []  # To store the 4-bit encoding for each image

for group_index, group in enumerate(groups):
    # Check if all amplitudes in the group have the same length
    group = [np.array(amplitudes) for amplitudes in group]  # Ensure each is a NumPy array

    # Calculate the Euclidean distance of each image in the group to a reference
    reference = group[0]

    distances = []
    for idx, amplitudes in enumerate(group):
        # Calculate Euclidean distance to the reference
        dist = np.linalg.norm(reference - amplitudes)
        distances.append((dist, idx))  # Store both distance and index

    # Step 3: Sort distances in ascending order and assign 4-bit codes
    distances_sorted = sorted(distances, key=lambda x: x[0])  # Sort by distance
    for i, (_, idx) in enumerate(distances_sorted):
        # Calculate the correct index in `file_names` for this group
        overall_index = group_index * 16 + idx
        # 4-bit code in binary from 0 (0000) to 15 (1111)
        binary_code = f"{i:04b}"
        am.append((file_names[overall_index], binary_code))  # Store image file name and its 4-bit encoding

# Output the results
# for image_name, code in am:
#     print(f"{image_name}: {code}")
# print(am)

[[[np.float64(92446228.83525425), np.float64(114675135.76885301), np.float64(153730802.91607094), np.float64(155943444.06828588), np.float64(43159721.80183371), np.float64(228118384.21785137), np.float64(283447090.4833941), np.float64(323302575.961489), np.float64(455557345.7854928), np.float64(502778040.00010175), np.float64(428215387.1247897), np.float64(529284428.8371862), np.float64(638690915.6877592), np.float64(585693421.1263492), np.float64(339713566.1733609), np.float64(697062285.0685332), np.float64(792174449.1108611), np.float64(879886576.6343734), np.float64(1226937496.5100608), np.float64(1239071870.5364697), np.float64(982342155.7301648), np.float64(1170321439.7613332), np.float64(1364779031.8896039), np.float64(1068426705.4335791)], [np.float64(83915293.65848988), np.float64(116153721.76041521), np.float64(138873259.20888588), np.float64(171714082.62327126), np.float64(37155798.94180141), np.float64(226753053.06511736), np.float64(255957818.26611692), np.float64(315827271

In [111]:
# Create a dictionary to store pm images for quick lookup
pm_dict = {img: code for img, code in pm}

# Merge pm and am into pm_am
pm_am = []
for img, code in am:
    if img in pm_dict:
        # Combine codes if the image is found in both pm and am
        pm_am.append((img, pm_dict[img], code))
    else:
        # If not found in pm, just add it from am
        pm_am.append((img, None, code))

# Optionally, if you want to ensure pm images are in the final output as well
for img, code in pm:
    if img not in [x[0] for x in pm_am]:
        pm_am.append((img, code, None))

# Sort pm_am based on the file name
pm_am_sorted = sorted(
    pm_am,
    key=lambda x: (
        int(x[0].split('_')[1].replace('prompt', '')),  # Extracting the number after 'prompt'
        int(x[0].split('_')[3].replace('image', '').replace('.png', ''))  # Extracting the number after 'image'
    )
)

# Display the sorted array
print(pm_am_sorted)

[('prompt_0_image_0.png', '0000', '0000'), ('prompt_0_image_1.png', '0000', '0010'), ('prompt_0_image_2.png', '0000', '1001'), ('prompt_0_image_3.png', '0000', '1110'), ('prompt_0_image_4.png', '0000', '1100'), ('prompt_0_image_5.png', '0000', '0111'), ('prompt_0_image_6.png', '0000', '0110'), ('prompt_0_image_7.png', '0000', '1011'), ('prompt_0_image_8.png', '0000', '1111'), ('prompt_0_image_9.png', '0000', '1000'), ('prompt_0_image_10.png', '0000', '1101'), ('prompt_0_image_11.png', '0000', '1010'), ('prompt_0_image_12.png', '0000', '0100'), ('prompt_0_image_13.png', '0000', '0011'), ('prompt_0_image_14.png', '0000', '0101'), ('prompt_0_image_15.png', '0000', '0001'), ('prompt_1_image_0.png', '0001', '0000'), ('prompt_1_image_1.png', '0001', '0101'), ('prompt_1_image_2.png', '0001', '1101'), ('prompt_1_image_3.png', '0001', '0111'), ('prompt_1_image_4.png', '0001', '0010'), ('prompt_1_image_5.png', '0001', '0100'), ('prompt_1_image_6.png', '0001', '0110'), ('prompt_1_image_7.png', '0

# Information Hiding

In [124]:
split_secret = [secret_in_binary[i:i + 8] for i in range(0, len(secret_in_binary), 8)]
split_secret_tuples = [(part[:4], part[4:]) for part in split_secret]

stego_images = []

for part1, part2 in split_secret_tuples:
    for prompt, val1, val2 in pm_am:
        if part1 == val1 and part2 == val2:
            stego_images.append((prompt, part1, part2))

print(stego_images)

[('prompt_4_image_9.png', '0100', '1000'), ('prompt_6_image_12.png', '0110', '0101'), ('prompt_6_image_9.png', '0110', '1100'), ('prompt_6_image_9.png', '0110', '1100'), ('prompt_6_image_13.png', '0110', '1111'), ('prompt_2_image_0.png', '0010', '0000'), ('prompt_5_image_7.png', '0101', '0111'), ('prompt_6_image_13.png', '0110', '1111'), ('prompt_7_image_7.png', '0111', '0010'), ('prompt_6_image_9.png', '0110', '1100'), ('prompt_6_image_5.png', '0110', '0100')]
